In [ ]:
# import libraries

from pyspark.sql.functions import substring, concat, col, lit
import pandas as pd
import numpy as np

In [ ]:
# Set storage name and file location

storage_account_name = "adlsazureproject2020"

file_location = "wasbs://datalake@adlsazureproject2020.blob.core.windows.net/BASE/StockPrices/EoD/20210731_EoDPrices.csv"
file_type = "csv"

In [ ]:
# Connect to Azure Storage using Key Vault
# https://docs.databricks.com/data/data-sources/azure/adls-gen2/azure-datalake-gen2-sp-access.html

spark.conf.set("fs.azure.account.auth.type." + storage_account_name + ".dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type." + storage_account_name + ".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id." + storage_account_name + ".dfs.core.windows.net", "8558216d-445c-49ae-b341-ff8146c6d539")
spark.conf.set("fs.azure.account.oauth2.client.secret." + storage_account_name + ".dfs.core.windows.net", dbutils.secrets.get(scope="key-vault-secrets",key="ServicePrincipleSecret"))
spark.conf.set("fs.azure.account.oauth2.client.endpoint." + storage_account_name + ".dfs.core.windows.net", "https://login.microsoftonline.com/75099764-499a-4c9a-be3b-eb20119a46dd/oauth2/token")

In [ ]:
# Load EoD prices into DF

dfEodPrices = spark.read.format(file_type).option("inferSchema", "true").option("header", "true").load(file_location)

In [ ]:
# Create extra columns

dfEodPrices = dfEodPrices.withColumn("ID", concat(substring('date', 6, 2),substring('date', 1, 4) ) ) #\
                          #.withColumn("DailyReturn", dfEodPrices['close'].pct_change(1))

#display(dfEodPrices)

In [ ]:
# Convert Spark dataframe to pandas for pct_change function use
dfEodPrices_pd = dfEodPrices.toPandas()

# Create new Daily Returns col
dfEodPrices_pd["DailyReturn"] = dfEodPrices_pd.groupby("Ticker")["close"].pct_change(1)

# Create Natural Log of daily returns
dfEodPrices_pd["LNDailyReturn"] = np.log(1 + dfEodPrices_pd["DailyReturn"])



In [ ]:
print(dfEodPrices_pd)

# will need to convert pandas df back to spark df. faster

date close high ... ID DailyReturn LNDailyReturn
0 1990-01-02 37.25 37.500 ... 011990 NaN NaN
1 1990-01-03 37.50 38.000 ... 011990 0.006711 0.006689
2 1990-01-04 37.63 38.750 ... 011990 0.003467 0.003461
3 1990-01-05 37.75 38.250 ... 011990 0.003189 0.003184
4 1990-01-08 38.00 38.000 ... 011990 0.006623 0.006601
... ... ... ... ... ... ... ...
15909 2021-07-26 289.05 289.690 ... 072021 -0.002140 -0.002143
15910 2021-07-27 286.54 289.575 ... 072021 -0.008684 -0.008722
15911 2021-07-28 286.22 290.150 ... 072021 -0.001117 -0.001117
15912 2021-07-29 286.50 288.618 ... 072021 0.000978 0.000978
15913 2021-07-30 284.91 286.660 ... 072021 -0.005550 -0.005565

[15914 rows x 17 columns]